# NLP Basic Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# 필요한 모듈 불러오기
import pandas as pd

In [34]:
%cd /content/drive/MyDrive/deeplearning

/content/drive/MyDrive/deeplearning


## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [35]:
spam = pd.read_csv('spam.csv')

In [36]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [38]:
spam.shape

(5572, 2)

## Tokenizing


In [104]:
import nltk

In [105]:
# 예시 코드 코드
from nltk.tokenize import word_tokenize
nltk.download('punkt')
word_tokenize(spam.iloc[5]['v2'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['FreeMsg',
 'Hey',
 'there',
 'darling',
 'it',
 "'s",
 'been',
 '3',
 'week',
 "'s",
 'now',
 'and',
 'no',
 'word',
 'back',
 '!',
 'I',
 "'d",
 'like',
 'some',
 'fun',
 'you',
 'up',
 'for',
 'it',
 'still',
 '?',
 'Tb',
 'ok',
 '!',
 'XxX',
 'std',
 'chgs',
 'to',
 'send',
 ',',
 'å£1.50',
 'to',
 'rcv']

In [106]:
spam['tk_sentence'] = spam.apply(lambda row: nltk.word_tokenize(row['v2']), axis=1)

In [107]:
spam.head()

,v1,v2,tk_sentence
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point, ,, crazy, .., Avail..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor, ..., U, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, do, n't, think, he, goes, to, usf, ,,..."


In [108]:
#원핫인코딩

corpus = set()
for sentence in spam['tk_sentence']:
    corpus.update(sentence)
corpus = pd.DataFrame(list(corpus))
pd.get_dummies(corpus)

'''
단어를 나타내기 위해 11519 차원의 벡터를 써야 한다는 것은 상당한 메모리 낭비다.
이에 아래의 Word Embedding 기법을 활용하겠다.
'''

'\n단어를 나타내기 위해 11519 차원의 벡터를 써야 한다는 것은 상당한 메모리 낭비다.\n이에 아래의 Word Embedding 기법을 활용하겠다.\n'

In [109]:
!pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

In [110]:
# 필요한 라이브러리들 올리기
# 사용할 라이브러리들이 import되어있어야 사용할 수 있습니다. 

import gensim
from gensim.models import Word2Vec, FastText
from glove import Corpus, Glove
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
corpus = spam['tk_sentence']
corpus.head()

0    [Go, until, jurong, point, ,, crazy, .., Avail...
1             [Ok, lar, ..., Joking, wif, u, oni, ...]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor, ..., U, c, alrea...
4    [Nah, I, do, n't, think, he, goes, to, usf, ,,...
Name: tk_sentence, dtype: object

## Word2Vec

In [69]:
#Skip-gram
wv_cb = Word2Vec(corpus, min_count = 5, size = 100, window = 2, iter = 200, sg = 0)
wv_sg = Word2Vec(corpus, min_count = 5, size = 100, window = 2, iter = 200, sg = 1)

In [70]:
wv_cb.save('w2v_cb')
wv_sg.save('w2v_sg')
w2v_cb = Word2Vec.load('w2v_cb')
w2v_sg = Word2Vec.load('w2v_sg')

In [73]:
w2v_cb.similarity('movie', 'see')

<ipython-input-73-4e9867b9301e>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  w2v_cb.similarity('movie', 'see')


-0.0059598098

In [74]:
w2v_sg.similarity('movie', 'see')

<ipython-input-74-a1d18e194a07>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  w2v_sg.similarity('movie', 'see')


0.150385

In [75]:
w2v_cb.similarity('movie', 'ticket')

<ipython-input-75-0f3b33ce1608>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  w2v_cb.similarity('movie', 'ticket')


0.13024682

In [77]:
w2v_sg.similarity('movie', 'ticket')

<ipython-input-77-7a6d4e1eae65>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  w2v_sg.similarity('movie', 'ticket')


0.22314718

In [95]:
w2v_cb.most_similar(positive=['movie'], topn=10)

<ipython-input-95-e8f505fb08c1>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_cb.most_similar(positive=['movie'], topn=10)


[('0870', 0.4610329866409302),
 ('5WB', 0.41685542464256287),
 ('kano', 0.3931582570075989),
 ('mode', 0.37799161672592163),
 ('kb', 0.37133944034576416),
 ('minute', 0.3660106956958771),
 ('tv', 0.36023983359336853),
 ('Landline', 0.34422561526298523),
 ('Croydon', 0.3400481343269348),
 ('Friendship', 0.33448660373687744)]

In [96]:
w2v_sg.most_similar(positive=['movie'], topn=10)

<ipython-input-96-08aa315a7fda>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_sg.most_similar(positive=['movie'], topn=10)


[('fantastic', 0.41554486751556396),
 ('0870', 0.3986990749835968),
 ('office', 0.3984825015068054),
 ('guaranteed', 0.3978520631790161),
 ('theatre', 0.38776224851608276),
 ('kb', 0.38281095027923584),
 ('bf', 0.3812482953071594),
 ('slow', 0.3785945177078247),
 ('sounds', 0.37666481733322144),
 ('K.k', 0.37643083930015564)]

In [80]:
w2v_cb.most_similar(positive=['crazy', 'gym'], negative= ['man'], topn=10)

<ipython-input-80-e6c0c6611666>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_cb.most_similar(positive=['crazy', 'gym'], negative= ['man'], topn=10)


[('ice', 0.3782382607460022),
 ('delivery', 0.36538463830947876),
 ('eve', 0.3549254536628723),
 ('auction', 0.3428378701210022),
 ('bill', 0.3410114645957947),
 ('Reply', 0.3214362561702728),
 ('8007', 0.3209819197654724),
 ('shopping', 0.3193211555480957),
 ('http', 0.3153645992279053),
 ('myself', 0.31443601846694946)]

In [81]:
w2v_sg.most_similar(positive=['crazy', 'gym'], negative= ['man'], topn=10)

<ipython-input-81-40010b8fb9fe>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_sg.most_similar(positive=['crazy', 'gym'], negative= ['man'], topn=10)


[('ice', 0.42895302176475525),
 ('comp', 0.3687867522239685),
 ('Tomorrow', 0.356628954410553),
 ('whatever', 0.3543817698955536),
 ('8007', 0.3534223437309265),
 ('Fancy', 0.3495965301990509),
 ('waking', 0.34870272874832153),
 ('TONE', 0.34230923652648926),
 ('NOKIA', 0.33800023794174194),
 ('pilates', 0.33496204018592834)]

In [82]:
w2v_cb.wv.doesnt_match('hotel movie see'.split())

/usr/local/lib/python3.8/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'hotel'

In [83]:
w2v_sg.wv.doesnt_match('hotel movie see'.split())

'hotel'

In [84]:
model = FastText(size=100, window=2)

In [87]:
model.build_vocab(corpus)

In [88]:
model.train(corpus, epochs = 200, 
            total_examples = model.corpus_count, 
            total_words = model.corpus_total_words)

In [89]:
print(model)

FastText(vocab=2150, size=100, alpha=0.025)


In [90]:
model.save('ft_model')
ft_model = FastText.load('ft_model')

In [91]:
ft_model.similarity('movie', 'see')

<ipython-input-91-f56992d7d781>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  ft_model.similarity('movie', 'see')


0.057544973

In [92]:
ft_model.similarity('movie', 'ticket')

<ipython-input-92-1e782fcc4a75>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  ft_model.similarity('movie', 'ticket')


0.07243433

In [97]:
ft_model.most_similar(positive=['movie'], topn=10)

<ipython-input-97-52f3313db99a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  ft_model.most_similar(positive=['movie'], topn=10)


[('movies', 0.7296054363250732),
 ('reason', 0.3493194580078125),
 ('mode', 0.34851136803627014),
 ('booked', 0.34083861112594604),
 ('move', 0.3400346040725708),
 ('sign', 0.33449041843414307),
 ('book', 0.3252932131290436),
 ('nite', 0.3203623294830322),
 ('ex', 0.31381338834762573),
 ('mrt', 0.3130566477775574)]

In [98]:
ft_model.doesnt_match('hotel movie see'.split())

<ipython-input-98-90e3dd9578b8>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  ft_model.doesnt_match('hotel movie see'.split())
/usr/local/lib/python3.8/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'hotel'

## GloVe

In [112]:
corPus = Corpus()
corPus.fit(corpus, window = 2)

In [117]:
model5 = Glove(no_components=5, learning_rate = 0.05)
model10 = Glove(no_components=10, learning_rate = 0.05)
model15 = Glove(no_components=15, learning_rate = 0.05)
model20 = Glove(no_components=15, learning_rate = 0.05)

model5.fit(corPus.matrix, epochs = 200, no_threads = 4, verbose = False)
model10.fit(corPus.matrix, epochs = 200, no_threads = 4, verbose = False)
model15.fit(corPus.matrix, epochs = 200, no_threads = 4, verbose = False)
model20.fit(corPus.matrix, epochs = 200, no_threads = 4, verbose = False)


In [118]:
model5.add_dictionary(corPus.dictionary)
model10.add_dictionary(corPus.dictionary)
model15.add_dictionary(corPus.dictionary)
model20.add_dictionary(corPus.dictionary)


In [119]:
model5.save('gv_model5')
model10.save('gv_model10')
model15.save('gv_model15')
model20.save('gv_model20')

gv_model5 = Glove.load('gv_model5')
gv_model10 = Glove.load('gv_model10')
gv_model15 = Glove.load('gv_model15')
gv_model20 = Glove.load('gv_model20')


In [123]:
gv_model5.most_similar('movie', 10)

[('Top', 0.9871927444204255),
 ('nice', 0.9757830037482578),
 ('else', 0.9742977579770828),
 ('thin', 0.9704362291905584),
 ("'RE", 0.9685899553588682),
 ('1956669', 0.961457650723103),
 ('1st', 0.9585824706919105),
 ('while', 0.9556759575040119),
 ('10am-7pm', 0.955660630173496)]

In [124]:
gv_model10.most_similar('movie', 10)

[('s.', 0.9101416181579829),
 ('09066362231', 0.8722131350269072),
 ('waste', 0.8664514131729659),
 ('First', 0.8598902906374971),
 ('girls', 0.839650727610254),
 ('chop', 0.8339501812337751),
 ('Gud', 0.8281268221451711),
 ('Got\\', 0.8250791603707965),
 ('urself', 0.8250351145056675)]

In [125]:
gv_model15.most_similar('movie', 10)

[('belong', 0.7857127335830257),
 ('Diwali', 0.778627194080653),
 ('laugh', 0.7741810178066513),
 ('friend', 0.7623505704117834),
 ('girls', 0.7586926763108448),
 ('fantastic', 0.7582625488428045),
 ('\\Happy', 0.7569189482794692),
 ('line', 0.7528254601103544),
 ('demand', 0.7477724094974215)]

In [126]:
gv_model20.most_similar('movie', 10)

[('6.Cruel', 0.7845278526355287),
 ('whole', 0.7669024002459081),
 ('girls', 0.756042844482221),
 ('laugh', 0.7522803727357406),
 ('second', 0.7337064689274742),
 ('Macha', 0.732868512014703),
 ('line', 0.717557136647909),
 ('friend', 0.716850502564981),
 ('big', 0.7125950477680346)]

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.

In [ ]:
'''
1. Word2Vec의 사용에 관하여
일반적으로 SkipGram을 통한 훈련이 CBOW를 통한 훈련보다 성능이 더 좋아짐을 확인했다.

2. WordEmbedding에 의해 측정되는 단어 간 유사도가
인간의 직관에 100% 들어맞는 것은 아니다.
이것이 Word Embedding 성능의 문제일 수도 있지만,
근본적으로 인간의 뇌가 작동하는 방식이 단어와 단어 사이의 '의미적 관계'를 중심으로 생각하는 반면
Word Embedding은 데이터에 의해 드러나는 수리적 frequency를 기준으로 측정되는 부분이 있을 것으로 생각된다.

3. GloVe의 사용에 관하여
일반적으로 n_components의 값이 클수록 인간의 직관과 유사한 결과가 나타남을 확인했다.
Dimension이 클수록 더 많은 정보를 참고할 수 있기에, 확률기반 모델이 인간 뇌가 참고하는 '의미적 관계'를
보다 더 정확히 추론하는 것으로 보인다.
그러나 이 데이터셋은 말뭉치의 크기가 비교적 작기 때문에,
더 큰 말뭉치를 사용할 경우우 오히려 너무 큰 Dimension이 성능을 저하할 가능성도 배제할 수 없다.
'''